In [ ]:
!pip install rasterio google-cloud-storage -q

# If running from orchestrator

import os
import re
import rasterio
from rasterio.merge import merge
from google.colab import auth
from google.cloud import storage
from tempfile import TemporaryDirectory
from google.api_core.exceptions import ServiceUnavailable, InternalServerError
from google.api_core.retry import Retry, if_exception_type

# ============================================================
# 🔑 Authenticate & Initialize
# ============================================================
auth.authenticate_user()
client = storage.Client()

bucket_name = "nasa_sar_spacetron"
carpeta_entrada = "CLEAN/SAR_Exports"      # where your .tif are
carpeta_salida  = "CLEAN/SAR_Mosaics"      # output folder
bucket = client.bucket(bucket_name)

# Define a retry strategy for uploads
# This strategy will retry on 503 (Service Unavailable) and 500 (Internal Server Error)
# errors with exponential backoff.
retry_strategy = Retry(
    predicate=if_exception_type(ServiceUnavailable, InternalServerError),
    initial=1.0,  # seconds
    maximum=60.0,  # seconds
    multiplier=2.0,
    total=600.0,  # seconds
)


# ============================================================
# 🗂️ List .tif files
# ============================================================
blobs = list(client.list_blobs(bucket_name, prefix=carpeta_entrada))
archivos_tif = [b.name for b in blobs if b.name.endswith(".tif")]
print(f"🗃️ {len(archivos_tif)} archivos encontrados en {carpeta_entrada}.\n")

# ============================================================
# 🧩 Group by (Year, Quartil) ignoring AOI number
# ============================================================
# Expected filenames: AOI_1_2020_Q1.tif
pattern = re.compile(r"AOI_\d+_(\d{4})_Q([1-4])")

grupos = {}
for path in archivos_tif:
    match = pattern.search(path)
    if match:
        year, q = match.groups()
        key = f"{year}_Q{q}"
        grupos.setdefault(key, []).append(path)

# Summary
for k, v in grupos.items():
    print(f"{k}: {len(v)} archivos (AOIs)")

# ============================================================
# 🧱 Mosaic all AOIs per Quartil-Year, skipping existing
# ============================================================
# List existing mosaics in the output folder
existing_mosaics_blobs = list(client.list_blobs(bucket_name, prefix=carpeta_salida))
existing_mosaics = {b.name for b in existing_mosaics_blobs}

with TemporaryDirectory() as tmpdir:
    for key, files in grupos.items():
        output_blob_name = f"{carpeta_salida}/AOI_ALL_{key}_mosaic.tif"
        if output_blob_name in existing_mosaics:
            print(f"⏩ {key}: Mosaico ya existe, se omite.")
            continue

        if len(files) < 2:
            print(f"⚠️ {key}: solo {len(files)} archivo(s), se omite.")
            continue

        print(f"\n🔄 Procesando mosaico {key} ({len(files)} AOIs)...")
        datasets = []

        # Download all AOI tiles
        for f in files:
            fname = os.path.join(tmpdir, os.path.basename(f))
            blob = bucket.blob(f)
            blob.download_to_filename(fname)
            src = rasterio.open(fname)
            datasets.append(src)

        # Merge all AOIs
        mosaic, out_transform = merge(datasets)

        # Copy metadata
        out_meta = datasets[0].meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_transform
        })

        # Save temporary file
        out_path_local = os.path.join(tmpdir, f"AOI_ALL_{key}_mosaic.tif")
        with rasterio.open(out_path_local, "w", **out_meta) as dest:
            dest.write(mosaic)

        # Upload to bucket with retry strategy
        out_blob = bucket.blob(output_blob_name)
        out_blob.upload_from_filename(out_path_local, retry=retry_strategy)
        print(f"✅ Subido: gs://{bucket_name}/{output_blob_name}")

print("\n🎯 Todos los mosaicos generados correctamente.")

🗃️ 384 archivos encontrados en CLEAN/SAR_Exports.

2018_Q1: 12 archivos (AOIs)
2018_Q2: 12 archivos (AOIs)
2018_Q3: 12 archivos (AOIs)
2018_Q4: 12 archivos (AOIs)
2019_Q1: 12 archivos (AOIs)
2019_Q2: 12 archivos (AOIs)
2019_Q3: 12 archivos (AOIs)
2019_Q4: 12 archivos (AOIs)
2020_Q1: 12 archivos (AOIs)
2020_Q2: 12 archivos (AOIs)
2020_Q3: 12 archivos (AOIs)
2020_Q4: 12 archivos (AOIs)
2021_Q1: 12 archivos (AOIs)
2021_Q2: 12 archivos (AOIs)
2021_Q3: 12 archivos (AOIs)
2021_Q4: 12 archivos (AOIs)
2022_Q1: 12 archivos (AOIs)
2022_Q2: 12 archivos (AOIs)
2022_Q3: 12 archivos (AOIs)
2022_Q4: 12 archivos (AOIs)
2023_Q1: 12 archivos (AOIs)
2023_Q2: 12 archivos (AOIs)
2023_Q3: 12 archivos (AOIs)
2023_Q4: 12 archivos (AOIs)
2024_Q1: 12 archivos (AOIs)
2024_Q2: 12 archivos (AOIs)
2024_Q3: 12 archivos (AOIs)
2024_Q4: 12 archivos (AOIs)
2025_Q1: 12 archivos (AOIs)
2025_Q2: 12 archivos (AOIs)
2025_Q3: 12 archivos (AOIs)
2025_Q4: 12 archivos (AOIs)
⏩ 2018_Q1: Mosaico ya existe, se omite.
⏩ 2018_Q2: Mo